# Run workflow using MLFlow

The workflow consists of an ETL and a Train part.

In [27]:
# Add the root of the repo to the sys path
import sys

sys.path[0] = "../"

In [28]:
import os
import mlflow
import core.mlflow_helper as mhelper
from core.util import load_config
from core.pandas_helper import load_pandas_df

In [29]:
# Setup the remote tracking server
general_conf = load_config("../workflow_conf.yaml", "general")
mhelper.set_remote_tracking(general_conf["experiment_name"])

### Run ETL

In [35]:
mlflow.run(
    "../",
    entry_point="etl",
    env_manager="local",
    parameters={},
)

2022/08/01 14:04:27 INFO mlflow.projects.docker: === Building docker image mlops_project:9fb140a ===
2022/08/01 14:04:38 INFO mlflow.projects.utils: === Created directory /tmp/tmpmrzouswr for downloading remote URIs passed to arguments of type 'path' ===
2022/08/01 14:04:38 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/subu/.aws:/.aws -e MLFLOW_RUN_ID=5728a35bf0a74bcbbf835e6f27fd502f -e MLFLOW_TRACKING_URI=http://10.100.55.198:5000 -e MLFLOW_EXPERIMENT_ID=5 docker.io/library/mlops_project:9fb140a python workflow_runner.py etl --no-run-on-delta --config-grid-index -1' in run with ID '5728a35bf0a74bcbbf835e6f27fd502f' === 


Getting all URLs
Loading URL content


100%|██████████| 116/116 [00:19<00:00,  5.98it/s]
[2022-08-01 14:05:00,896] [INFO] [botocore.credentials]: Found credentials in shared credentials file: ~/.aws/credentials
2022/08/01 14:05:01 INFO mlflow.projects: === Run (ID '5728a35bf0a74bcbbf835e6f27fd502f') succeeded ===


### Run training

In [36]:
mlflow.run(
    "../",
    entry_point="train",
    parameters={"input_data_run_id_arg":"5728a35bf0a74bcbbf835e6f27fd502f"},
)

2022/08/01 14:07:04 INFO mlflow.projects.docker: === Building docker image mlops_project:9fb140a ===
2022/08/01 14:07:14 INFO mlflow.projects.utils: === Created directory /tmp/tmp38v_e3bh for downloading remote URIs passed to arguments of type 'path' ===
2022/08/01 14:07:14 INFO mlflow.projects.backend.local: === Running command 'docker run --rm -v /home/subu/.aws:/.aws -e MLFLOW_RUN_ID=52b907aeac10486cbdb860feb3fed4e4 -e MLFLOW_TRACKING_URI=http://10.100.55.198:5000 -e MLFLOW_EXPERIMENT_ID=5 docker.io/library/mlops_project:9fb140a python workflow_runner.py train 5728a35bf0a74bcbbf835e6f27fd502f --config-grid-index -1' in run with ID '52b907aeac10486cbdb860feb3fed4e4' === 
[2022-08-01 14:07:17,164] [INFO] [core.worker]: Starting MLFlow experiment for training
[2022-08-01 14:07:17,640] [INFO] [botocore.credentials]: Found credentials in shared credentials file: ~/.aws/credentials
2022/08/01 14:07:18 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/